In [1]:
import datajoint as dj
import numpy as np
import time
import pymeshfix

dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config["display.limit"] = 20
    
#schema = dj.schema('microns_ta3p100')
#ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')
schema = dj.schema("microns_pinky")
pinky = dj.create_virtual_module("pinky","microns_pinky")

Connecting celiib@10.28.0.34:3306


In [2]:
@schema
class PymeshfixDecimatedExcitatoryStitchedMesh(dj.Computed):
    definition = """
    -> pinky.Decimation35ExcitatoryStitchedMesh               
    ---
    n_vertices           : bigint  #total number of vertices after pymeshfix cleaning                     
    n_triangles          : bigint  #total number of faces after pymeshfix cleaning                    
    vertices             : longblob #vertices after pymeshfix cleaning                  
    triangles            : longblob #faces after pymeshfix cleaning  
    perc_vertices_remaining  : decimal(6,5) #number of vertices still present after pymeshfix cleaning
    perc_triangles_remaining  : decimal(6,5) #number of faces still present after pymeshfix cleaning
    
    """
    
    #getting the keys from the following three tables
    #key_source = ta3p100.Mesh & ta3p100.CurrentSegmentation & ta3p100.AllenSomaClass
    
    def make(self, key):
        start_time = time.time()
        
        print("\n\n*****Starting pymeshfix algorithm for " + str(key["segment_id"]) + "******")
        
        joincomp = True
        remove_smallest_components = False
        
        
        #get the vertices 
        mesh = (pinky.Decimation35ExcitatoryStitchedMesh & key).fetch1()
        vertices, triangles = mesh['vertices'], mesh['triangles']
        meshfix = pymeshfix.MeshFix(vertices,triangles)
        verbose=False
        meshfix.repair(verbose,joincomp,remove_smallest_components)
        new_verts = meshfix.v
        new_faces = meshfix.f

        
        self.insert1(dict(key,
                          n_vertices=len(new_verts),
                          n_triangles=len(new_faces),
                          vertices=new_verts,
                          triangles=new_faces,
                         perc_vertices_remaining=len(new_verts)/len(vertices),
                         perc_triangles_remaining=len(new_faces)/len(triangles)),skip_duplicates=True)
        
        
        print(f"Finished pymeshfix algorithm: {time.time() - start_time}")
        


In [3]:
start = time.time()
PymeshfixDecimatedExcitatoryStitchedMesh.populate(reserve_jobs=True)
print(time.time() - start)



*****Starting pymeshfix algorithm for 648518346349472601******
Finished pymeshfix algorithm: 95.60092830657959


*****Starting pymeshfix algorithm for 648518346349476961******
Finished pymeshfix algorithm: 177.22993636131287


*****Starting pymeshfix algorithm for 648518346349491311******
Finished pymeshfix algorithm: 70.53941869735718


*****Starting pymeshfix algorithm for 648518346349493881******
Finished pymeshfix algorithm: 91.60535001754761


*****Starting pymeshfix algorithm for 648518346349496925******
Finished pymeshfix algorithm: 135.5162136554718


*****Starting pymeshfix algorithm for 648518346349499803******
Finished pymeshfix algorithm: 114.40877556800842


*****Starting pymeshfix algorithm for 648518346349500440******
Finished pymeshfix algorithm: 168.65908432006836


*****Starting pymeshfix algorithm for 648518346349503924******
Finished pymeshfix algorithm: 226.74040341377258


*****Starting pymeshfix algorithm for 648518346349507226******
Finished pymeshfix algorith

In [4]:
#(schema.jobs & "table_name='__pymeshfix_decimated_excitatory_stitched_mesh'").delete()

In [6]:
pinky.PymeshfixDecimatedExcitatoryStitchedMesh() & "perc_triangles_remaining < 0.95"

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,n_vertices total number of vertices after pymeshfix cleaning,n_triangles total number of faces after pymeshfix cleaning,vertices vertices after pymeshfix cleaning,triangles faces after pymeshfix cleaning,perc_vertices_remaining number of vertices still present after pymeshfix cleaning,perc_triangles_remaining number of faces still present after pymeshfix cleaning
3,648518346349470171,0.35,352926,706364,=BLOB=,=BLOB=,0.91522,0.91810
3,648518346349471156,0.35,257594,515604,=BLOB=,=BLOB=,0.94457,0.94117
3,648518346349471500,0.35,232394,465152,=BLOB=,=BLOB=,0.90327,0.89243
3,648518346349471910,0.35,659301,1320374,=BLOB=,=BLOB=,0.94923,0.94471
3,648518346349472574,0.35,524294,1049448,=BLOB=,=BLOB=,0.94572,0.93933
3,648518346349473583,0.35,153274,306848,=BLOB=,=BLOB=,0.81161,0.79979
3,648518346349473821,0.35,238044,476724,=BLOB=,=BLOB=,0.93222,0.92695
3,648518346349473830,0.35,74313,148738,=BLOB=,=BLOB=,0.75325,0.74039
3,648518346349473833,0.35,156746,313864,=BLOB=,=BLOB=,0.91021,0.90138
3,648518346349473835,0.35,137155,274874,=BLOB=,=BLOB=,0.88429,0.86368
